In [1]:
from tqdm.auto import tqdm
import torch
import mlflow

from parameters import MODEL_NAME

In [2]:
# Определение устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
import os
from dotenv import load_dotenv
from track_model import init_mlflow

load_dotenv()
init_mlflow()

MLFlow URI: http://188.72.77.22:5050/


In [4]:
from seed_initializer import seed_all
seed_all()

In [5]:
mlflow_run = mlflow.start_run(log_system_metrics=True)

2025/06/01 20:31:29 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/01 20:31:29 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


In [6]:
from data_loader import get_data_bundle
data_bundle = get_data_bundle(project_path="", num_workers=4)

Dataset Sizes:
  Train: 8330
  Val: 1041
  Test: 624

Class Information:
  Class: NORMAL, Weight: 0.7411, Count: 2157
  Class: PNEUMONIA, Weight: 0.2589, Count: 6173

Batch dimension example:
  torch.Size([32, 3, 224, 224])


In [7]:
from parameters import USE_GPU

from train_model import train_model
model, train_loss, train_acc, val_loss, val_acc = train_model(
            proj_path="",
            data_bundle=data_bundle,
            device=device,
            model_name=MODEL_NAME,
            dry_run=not USE_GPU,
        )


Epoch 1/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 2/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 3/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 4/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 5/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 6/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 7/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.00%
  Epoch Time: 0.00 seconds


Epoch 8/10 metrics:
  Train Loss: 0.0000 | Train Acc: 0.00%
  Val Loss:   0.0000 | Val Acc:   0.

#Функции обучения и оценки

In [8]:
from evaluate_model import evaluate_model
y_true, y_pred, y_scores = evaluate_model(model, data_bundle.loaders.test)

Оценка модели: 100%|██████████| 20/20 [01:55<00:00,  5.78s/it]


In [9]:
from track_model import log_model_as_onnx
log_model_as_onnx(model, make_current=True)

Сохраняем модель в формате ONNX: /tmp/medical-image-processing/hybrid_model_455e0b29a18647a89d09fee2d60eb8b7.onnx


/home/alina/medical-image-processing/xray-classifier/.venv/lib/python3.12/site-packages/transformers/models/vit/modeling_vit.py:154: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/home/alina/medical-image-processing/xray-classifier/.venv/lib/python3.12/site-packages/transformers/models/vit/modeling_vit.py:160: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if height != self.image_size[0] or width != self.image_size[1]:


Загружаем модель в MLflow: /tmp/medical-image-processing/hybrid_model_455e0b29a18647a89d09fee2d60eb8b7.onnx


Registered model 'xray-hybrid-classifier' already exists. Creating a new version of this model...
2025/06/01 20:34:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xray-hybrid-classifier, version 2
Created version '2' of model 'xray-hybrid-classifier'.


Удаляем временный файл с моделью: /tmp/medical-image-processing/hybrid_model_455e0b29a18647a89d09fee2d60eb8b7.onnx


In [10]:
mlflow.end_run()

🏃 View run dapper-snake-53 at: http://188.72.77.22:5050/#/experiments/1/runs/455e0b29a18647a89d09fee2d60eb8b7
🧪 View experiment at: http://188.72.77.22:5050/#/experiments/1


2025/06/01 20:35:18 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/06/01 20:35:18 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

classes = ['NORMAL', 'PNEUMONIA']



# Подробный отчет о классификации
print("ДЕТАЛЬНЫЙ ОТЧЕТ О КЛАССИФИКАЦИИ:")
print(classification_report(y_true, y_pred, target_names=classes, digits=4))

# Матрица ошибок
cm = confusion_matrix(y_true, y_pred)
print("\nМАТРИЦА ОШИБОК:")
print(cm)

# Визуализация матрицы ошибок
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Предсказанные метки")
plt.ylabel("Истинные метки")
plt.title("Матрица ошибок")
plt.show()

# ROC-AUC
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# Визуализация ROC-кривой
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC AUC = {roc_auc:.3f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривая")
plt.legend(loc="lower right")
plt.show()